In [1]:
from importlib import reload
import sys
sys.path.insert(0, '../')

# sys.path.remove("/home/users/yixiuz/.local/lib/python3.9/site-packages")
sys.path.append("/home/groups/swl1/yixiuz/torch_fid/lib/python3.9/site-packages")
sys.path.append("/home/groups/swl1/yixiuz/torch_fid/bin")

In [2]:
import matplotlib
import torch
import torch.nn as nn
import torch.nn.functional as F
import ml_collections

import config.eval.countdown as eval_config
# import config.eval.genes_base as eval_config
# import config.eval.genes_hollow as eval_config

import lib.utils.bookkeeping as bookkeeping
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

import lib.utils.utils as utils
import lib.models.models as models
import lib.models.model_utils as model_utils
import lib.datasets.datasets as datasets
import lib.datasets.dataset_utils as dataset_utils
import lib.sampling.sampling as sampling
import lib.sampling.sampling_utils as sampling_utils

%matplotlib inline

eval_cfg = eval_config.get_config()
train_cfg = bookkeeping.load_ml_collections(Path(eval_cfg.train_config_path))

for item in eval_cfg.train_config_overrides:
    utils.set_in_nested_dict(train_cfg, item[0], item[1])

S = train_cfg.data.S
device = torch.device(eval_cfg.device)
# device = torch.device('cpu')

model = model_utils.create_model(train_cfg, device)

loaded_state = torch.load(Path(eval_cfg.checkpoint_path),
    map_location=device)

modified_model_state = utils.remove_module_from_keys(loaded_state['model'])
model.load_state_dict(modified_model_state)

model.eval()

dataset = dataset_utils.get_dataset(eval_cfg, device)
data = dataset.data

2024-05-20 19:17:10.429249: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 19:17:10.598355: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /share/software/user/open/cudnn/8.9.0.131/lib:/usr/lib64/nvidia:/share/software/user/open/cuda/12.2.0/targets/x86_64-linux/lib:/share/software/user/open/cuda/12.2.0/lib64:/share/software/user/open/cuda/12.2.0/nvvm/lib64:/share/software/user/open/cuda/12.2.0/extras/Debugger/lib64:/share/software/user/open/cuda/12.2.0/extras/CUPTI/lib64:/share/software/user/open/python/3.9.0/lib:/share/software/u

In [3]:
from eval import count_mistakes, get_dist, hellinger, eval_mse_stats, save_results
from tqdm import tqdm

In [8]:
def run_experiments(num_repeats, sample_size, results_file,
                   method, g_steps, tau_steps, 
                    c_steps, c_stepsize, corrector):
    tqdm._instances.clear()

    # Tau-leaping only
    eval_cfg.sampler.num_steps = tau_steps

    # Generic corrector fields
    eval_cfg.sampler.num_corrector_steps = c_steps
    eval_cfg.sampler.corrector_entry_time = 0.9
    eval_cfg.sampler.corrector_step_size_multiplier = c_stepsize
    eval_cfg.sampler.balancing_function = corrector

    if method == "gillespies":
        eval_cfg.sampler.name = "PCMultiGillespies"
    elif method == "tauleaping":
        eval_cfg.sampler.name = "PCTauLeapingAbsorbingInformed"
    else:
        assert(False)

    sampler = sampling_utils.get_sampler(eval_cfg)

    results = []

    for _ in range(num_repeats):
        if method == "gillespies":
            samples, out = sampler.sample(model, sample_size, updates_per_eval=g_steps)
        elif method == "tauleaping":
            samples, out = sampler.sample(model, sample_size, 0)

        data_flat = dataset.data.flatten().cpu().numpy()
        h_dist = hellinger(data_flat, samples.flatten(), eval_cfg.data.S)
        error_rate = count_mistakes(samples)

        D = eval_cfg.data.shape[0]
        if method == "gillespies":
            nfe = D / g_steps
        elif method == "tauleaping":
            nfe = eval_cfg.sampler.num_steps
        nfe += nfe * eval_cfg.sampler.corrector_entry_time * eval_cfg.sampler.num_corrector_steps

        new_result = {
                    'method': method,
                    'g_steps': 0 if method != "gillespies" else g_steps,
                    'tau_steps': 0 if method != "tauleaping" else eval_cfg.sampler.num_steps,
                    'use_corrector': eval_cfg.sampler.corrector_entry_time > 0 
                                 and eval_cfg.sampler.num_corrector_steps > 0,
                    'corrector': eval_cfg.sampler.balancing_function,
                    'c_stepsize': eval_cfg.sampler.corrector_step_size_multiplier,
                    'c_steps': eval_cfg.sampler.num_corrector_steps,
                    'nfe': nfe,
                    'h_dist': h_dist,
                    'error_rate': error_rate,
                }
        print(new_result)
        results.append(new_result)

    save_results(results, results_file)

In [10]:
results_file = 'countdown_results_test_1.csv'

num_repeats = 2
sample_size = 10

methods = ["tauleaping", 
           "gillespies"]

updates_per_evals = [3, 2, 1]
num_sample_steps = [50, 100, 200]

correctors = ["mpf", "barker", "birthdeath"]
corrector_stepsizes = [2.0, 1.0, 0.1]
corrector_steps = 2

for method in methods:
    for i in range(len(correctors)):
        corrector = correctors[i]
        c_stepsize = corrector_stepsizes[i]
        g_steps = updates_per_evals[0]
        tau_steps = num_sample_steps[0]
        c_steps = corrector_steps
        run_experiments(num_repeats=num_repeats, 
                        sample_size=sample_size, 
                        results_file=results_file,
                        method=method, 
                        g_steps=g_steps, 
                        tau_steps=tau_steps, 
                        c_steps=c_steps, 
                        c_stepsize=c_stepsize, 
                        corrector=corrector)

49it [00:01, 28.44it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 2.0, 'c_steps': 2, 'nfe': 140.0, 'h_dist': 0.11997139312547875, 'error_rate': 0.001953125}


49it [00:01, 28.41it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 2.0, 'c_steps': 2, 'nfe': 140.0, 'h_dist': 0.12998509095246613, 'error_rate': 0.0015625}
Experiment results saved to  countdown_results_test_1.csv


49it [00:01, 28.32it/s]


...

{'method': 'gillespies', 'g_steps': 3, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'barker', 'c_stepsize': 1.0, 'c_steps': 2, 'nfe': 238.93333333333334, 'h_dist': 0.03361838294970329, 'error_rate': 0.0171875}
Experiment results saved to  countdown_results_test_1.csv


258it [00:03, 81.36it/s]                         


{'method': 'gillespies', 'g_steps': 3, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 238.93333333333334, 'h_dist': 0.037362955891120334, 'error_rate': 0.0234375}


258it [00:03, 81.63it/s]                         


{'method': 'gillespies', 'g_steps': 3, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 238.93333333333334, 'h_dist': 0.029003809342050917, 'error_rate': 0.028515625}
Experiment results saved to  countdown_results_test_1.csv


In [11]:
results_file = 'countdown_results.csv'

num_repeats = 5
sample_size = 500

methods = ["tauleaping", 
           "gillespies"]

updates_per_evals = [3, 2, 1]
num_sample_steps = [50, 100, 200]

correctors = ["mpf", "barker", "birthdeath"]
corrector_stepsizes = [2.0, 1.0, 0.1]
corrector_steps = 1

# 1 corrector step
c_steps = 1
for method in methods:
    for i in range(len(correctors)):
        
        corrector = correctors[i]
        c_stepsize = corrector_stepsizes[i]
        
        for j in range(len(updates_per_evals)):
        
            g_steps = updates_per_evals[j]
            tau_steps = num_sample_steps[j]
            
            run_experiments(num_repeats=num_repeats, 
                            sample_size=sample_size, 
                            results_file=results_file,
                            method=method, 
                            g_steps=g_steps, 
                            tau_steps=tau_steps, 
                            c_steps=c_steps, 
                            c_stepsize=c_stepsize, 
                            corrector=corrector)

49it [00:21,  2.33it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 2.0, 'c_steps': 1, 'nfe': 95.0, 'h_dist': 0.11900228366884924, 'error_rate': 0.0047421875}


49it [00:21,  2.33it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 2.0, 'c_steps': 1, 'nfe': 95.0, 'h_dist': 0.11448413026773421, 'error_rate': 0.0041875}


49it [00:21,  2.33it/s]


...

{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 1, 'nfe': 486.4, 'h_dist': 0.02218616772837191, 'error_rate': 0.0098984375}


100%|██████████| 256/256 [01:51<00:00,  2.29it/s]


{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 1, 'nfe': 486.4, 'h_dist': 0.02146416473464358, 'error_rate': 0.0098515625}


100%|██████████| 256/256 [01:51<00:00,  2.29it/s]


{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 1, 'nfe': 486.4, 'h_dist': 0.023039434467958836, 'error_rate': 0.0094296875}
Experiment results saved to  countdown_results.csv


In [12]:
# 2 corrector steps
c_steps = 2
for method in methods:
    for i in range(len(correctors)):
        
        corrector = correctors[i]
        c_stepsize = corrector_stepsizes[i]
        
        for j in range(len(updates_per_evals)):
        
            g_steps = updates_per_evals[j]
            tau_steps = num_sample_steps[j]
            
            run_experiments(num_repeats=num_repeats, 
                            sample_size=sample_size, 
                            results_file=results_file,
                            method=method, 
                            g_steps=g_steps, 
                            tau_steps=tau_steps, 
                            c_steps=c_steps, 
                            c_stepsize=c_stepsize, 
                            corrector=corrector)

49it [00:30,  1.59it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 2.0, 'c_steps': 2, 'nfe': 140.0, 'h_dist': 0.12614697877938105, 'error_rate': 0.00275}


49it [00:30,  1.59it/s]


{'method': 'tauleaping', 'g_steps': 0, 'tau_steps': 50, 'use_corrector': True, 'corrector': 'mpf', 'c_stepsize': 2.0, 'c_steps': 2, 'nfe': 140.0, 'h_dist': 0.1211698216426712, 'error_rate': 0.002828125}


49it [00:30,  1.59it/s]


...

{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 716.8, 'h_dist': 0.023546523004305688, 'error_rate': 0.009265625}


100%|██████████| 256/256 [02:44<00:00,  1.56it/s]


{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 716.8, 'h_dist': 0.023155009461076275, 'error_rate': 0.008796875}


100%|██████████| 256/256 [02:44<00:00,  1.56it/s]


{'method': 'gillespies', 'g_steps': 1, 'tau_steps': 0, 'use_corrector': True, 'corrector': 'birthdeath', 'c_stepsize': 0.1, 'c_steps': 2, 'nfe': 716.8, 'h_dist': 0.02286865058675308, 'error_rate': 0.009453125}
Experiment results saved to  countdown_results.csv
